Some FGS images made by MIRAGE show very wide spreads in count rates. Looking into this, it looks like there are a few highly negative pixels and a few (or one) highly positive pixel. When MAGIC tries to normalize this image, it throws all of the available counts into the one (or couple) highly positive pixel(s). This means that we have no PSF data in the converted image. 

In my analysis I was able to come up with a quick fix using the correct_image utility function in MAGIC that resets all negative values to zero and all values over a given threshold to that threshold. 

In MAGIC we **do not want to cap the input image unless it has gone through the pipeline** (since seed images will have count rates well into the millions for certain stars) or if we know it is a simulated image. 


In [ ]:
# Import necessary packages
import os 

from astropy.io import fits

from jwst_magic.utils import utils

In [ ]:
# Path to the file that needs to be changed 
 
path_to_file = 'path_to_the_file_to_be_corrected' # CHANGE THIS VARIABLE 


In [ ]:
# Open the file and print out information about the file

hdu = fits.open(path_to_file)
hdu.info()


In [ ]:
# Based on the info that is printed above, choose the extension(s) of the hdu to correct
# For the extension that has the image use either the associated "No." or "Name" in the 
#  above info for the variable `extension` below 
extension = 'primary'
data = hdu[extension].data

In [ ]:
# Correct the pixels to make sure all count rates fall within 0 and 65,000
new_data = utils.correct_image(data, upper_threshold=65000, upper_limit=65000)

hdu[extension].data = new_data

In [ ]:
# Give the new filename which is in the same location as the old file

path, filename = os.path.split(path_to_file)
parts = filename.split('.')
new_filename = f"{parts[0]}_corrected.{parts[1]}"
new_file = os.path.join(path, new_filename)

print(new_file)

In [ ]:
# Write out updated data to a new file that will be used as input into MAGIC

hdu.writeto(new_file)